<a href="https://colab.research.google.com/github/Bitdribble/ldl/blob/main/colab/c6e2_boston_conf6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


This code example is very similar to c6e1_boston but the network is modified to use more layers with more neurons as well as dropout regularization. More context for this code example can be found in the section "Experiment: Deeper and Regularized Models for House Price Prediction" in Chapter 6 in the book Learning Deep Learning by Magnus Ekman (ISBN: 9780137470358).

One thing to note is that Dropout layers do behave differently between training and test. Therefore it is crucial that the training loop calls model.train() and model.eval() correctly. However, given that we started doing that in our very first PyTorch example, our training loop already does this and no changes are needed.


In [4]:
# Set up sandbox environment
!rm -rf ldl
!git clone https://github.com/Bitdribble/ldl.git

# Install module dependencies
!pip install -r /content/ldl/colab_requirements.txt

%cd /content/ldl

Cloning into 'ldl'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 315 (delta 196), reused 205 (delta 99), pack-reused 0
Receiving objects: 100% (315/315), 1.32 MiB | 4.27 MiB/s, done.
Resolving deltas: 100% (196/196), done.
/content/ldl


In [7]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from pt_framework.utilities import train_model_df

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 500
BATCH_SIZE = 16

# Read and standardize the data.
boston_housing = load_boston()
data = boston_housing.get('data')
target = boston_housing.get('target')

raw_x_train, raw_x_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=0)

# Convert to same precision as model.
raw_x_train = raw_x_train.astype(np.float32)
raw_x_test = raw_x_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))

x_mean = np.mean(raw_x_train, axis=0)
x_stddev = np.std(raw_x_train, axis=0)
x_train = (raw_x_train - x_mean) / x_stddev
x_test = (raw_x_test - x_mean) / x_stddev

# Create Dataset objects.
trainset = TensorDataset(torch.from_numpy(x_train),
                         torch.from_numpy(y_train))
testset = TensorDataset(torch.from_numpy(x_test),
                        torch.from_numpy(y_test))

# Create model.
model = nn.Sequential(
    nn.Linear(13, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 1)
)

# Initialize weights.
for module in model.modules():
    if isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)
        nn.init.constant_(module.bias, 0.0)

# Loss function and optimizer
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.MSELoss()

# Train model.
return_value = train_model_df(model, device, EPOCHS, BATCH_SIZE, trainset, testset,
                              optimizer, loss_function, 'mae')




/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

Epoch 1/500 loss: 526.9859 - mae: 20.4366 - val_loss: 378.0029 - val_mae: 15.9264
Epoch 2/500 loss: 209.7262 - mae: 11.4133 - val_loss: 90.0263 - val_mae: 6.6252
Epoch 3/500 loss: 70.8487 - mae: 6.1850 - val_loss: 55.8946 - val_mae: 4.9102
Epoch 4/500 loss: 45.5957 - mae: 4.9915 - val_loss: 39.0017 - val_mae: 3.9110
Epoch 5/500 loss: 36.5093 - mae: 4.4397 - val_loss: 33.0949 - val_mae: 3.6348
Epoch 6/500 loss: 36.3247 - mae: 4.4348 - val_loss: 29.8222 - val_mae: 3.3960
Epoch 7/500 loss: 38.2134 - mae: 4.3976 - val_loss: 32.7308 - val_mae: 3.5059
Epoch 8/500 loss: 36.0591 - mae: 4.3355 - val_loss: 33.2369 - val_mae: 3.5494
Epoch 9/500 loss: 28.7799 - mae: 4.0381 - val_loss: 28.2441 - val_mae: 3.2017
Epoch 10/500 loss: 32.1692 - mae: 4.0872 - val_loss: 28.7273 - val_mae: 3.2479
Epoch 11/500 loss: 29.3949 - mae: 4.0135 - val_loss: 28.9720 - val_mae: 3.2662
Epoch 12/500 loss: 33.8734 - mae: 4.0563 - val_loss: 26.9750 - val_mae: 3.1177
Epoch 13/500 loss: 35.9305 - mae: 4.3173 - val_loss: 27

In [8]:
# Display the loss and error chart
import altair as alt

# Operate on a copy to not modify loss_df
df = return_value["df"].copy()
data = df.reset_index().melt(
    id_vars=['epoch'], 
    value_vars=['loss', 'mae', 'val_loss', 'val_mae'])

bind = alt.selection_interval(bind='scales')

alt.Chart(data).mark_line().encode(
    x='epoch',
    y='value',
    color='variable'
).add_selection(
    bind
)

alt.Chart(...)

In [9]:
# Print first 4 predictions.
inputs = torch.from_numpy(x_test)
inputs = inputs.to(device)
outputs = model(inputs)
for i in range(0, 4):
    print('Prediction: %4.2f' % outputs.data[i].item(),
         ', true value: %4.2f' % y_test[i].item())

Prediction: 23.87 , true value: 22.60
Prediction: 29.04 , true value: 50.00
Prediction: 24.50 , true value: 23.00
Prediction: 9.24 , true value: 8.30
